In [1]:
%serialconnect --port='/dev/ttyS15'

Connecting to --port=/dev/ttyS15 --baud=115200 
Ready.


In [2]:
%rebootdevice

repl is in normal command mode
[\r\x03\x03] b'\r\nMicroPython v1.9.4-773-gafecc124e on 2019-01-09; ESP32 module with ESP32\r\nType "help()" for more information.\r\n>>> \r\n>>> \r\nPYB: soft reboot\r\nOSError: [Errno 2] ENOENT\r\nMicroPython v1.9.4-773-gafecc124e on 2019-01-09; ESP32 module with ESP32\r\nType "help()" for more information.\r\n>>> \r\n>>> \r\n>>> '
[\r\x01] b'\r\n>>> \r\nraw REPL; CTRL-B to exit\r\n>'

In [3]:
import machine
import utime
import ustruct

# List Important Registers
Sourced from [datasheet](https://statics.cirrus.com/pubs/proDatasheet/CS5490_F3.pdf)

In [4]:
# HARDWARE Page 0
HARDWARE_PAGE = 0x80

# List Addresses
CONFIG0 = 0x00
CONFIG1 = 0x01
MASK = 0x03
PC = 0x05
SERIALCTRL = 0x07
PULSEWIDTH = 0x08
PULSECTRL = 0x09
STATUS0 = 0x17
STATUS1 = 0x18
STATUS2 = 0x19
REGLOCK = 0x22
VPEAK = 0x24
IPEAK = 0x25
PSDC = 0x30
ZXNUM = 0x37

# SOFTWARE Page 16
SOFTWARE_P16 = 0x90

# List Addresses
CONFIG2 = 0x00
REGCHK = 0x01
V_INST = 0x02
I_INST = 0x03
P_INST = 0x04
P_AVG = 0x05
I_RMS = 0x06
V_RMS = 0x07
Q_AVG = 0x0E
Q_INST = 0x0F
S_INST = 0x14
PF = 0x15
TEMP = 0x1B
P_SUM = 0x1D
S_SUM = 0x1E
Q_SUM = 0x1F
I_DCOFF = 0x20
I_GAIN = 0x21
V_DCOFF = 0x22
V_GAIN = 0x23
P_OFF = 0x24
I_ACOFF = 0x25
EPSILON = 0x31
SAMPLECOUNT = 0x33
T_GAIN = 0x36
T_OFF = 0x37
T_SETTLE = 0x39
LOAD_MIN = 0x40
SYS_GAIN = 0x3C
SYS_TIME = 0x3D

# SOFTWARE Page 17
SOFTWARE_P17 = 0x91

# List Addresses
VSAG_DUR = 0x00
VSAG_LEVEL = 0x01
IOVER_DUR = 0x04
IOVER_LEVEL = 0x05

# SOFTWARE Page 18
SOFTWARE_P18 = 0x92

# List Addresses
IZX_LEVEL = 0x18
PULSERATE = 0x1C
INT_GAIN = 0x2B
VSWELL_DUR = 0x2E
VSWELL_LEVEL = 0x2F
VZX_LEVEL = 0x3A
CYCLECOUNT = 0x3E
SCALE = 0x3F

# Instructions (pre masked with instruction byte prefix 0b11000000)

# Controls
SOFT_RESET = 0xC1
STANDBY = 0xC2
WAKEUP = 0xC3
SINGLE_CONV =  0xD4
CONT_CONV = 0xD5
HALT_CONV = 0xD8

# Calibration
I_AC_CAL = 0xF1
V_AC_CAL = 0xF2
IV_AC_CAL = 0xF6
I_DC_CAL = 0xE1
V_DC_CAL = 0xE2
IV_DC_CAL = 0xE6
I_GAIN_CAL = 0xF9
V_GAIN_CAL = 0xFA
IV_GAIN_CAL = 0xFE

In [5]:
u2 = machine.UART(2, baudrate=600, rx=16, tx=17, timeout=20)
utime.sleep(1)

In [6]:
def read_register(page,register):
    u2.write(bytearray([page,register]))
    utime.sleep(1)
    buffer = bytearray(4)
    buffer[1:] = u2.read(3)
    val = ustruct.unpack('>L',buffer)[0]
    return val

In [7]:
def write_register(page,register,val):
    register = 1<<6 | register
    u2.write(bytearray([page,register]))
    buffer = ustruct.pack('>BBB',(val>>16)&0xff,(val>>8)&0xff,val&0xff)
    u2.write(buffer)

In [8]:
def instruction(inst_byte):
    u2.write(bytearray([inst_byte]))

# Persorm Soft RESET

In [9]:
instruction(SOFT_RESET)

# Read Hardware register Config0

In [10]:
print(read_register(HARDWARE_PAGE,CONFIG0))

8384


# Read Software register Voltage Gain, Current Gain, Temperature Gain

In [11]:
print(read_register(SOFTWARE_P16,V_GAIN))

64


In [12]:
write_register(SOFTWARE_P16,V_GAIN,8192)

In [13]:
print(read_register(SOFTWARE_P16,V_GAIN))

8192


In [14]:
print(read_register(SOFTWARE_P16,I_GAIN))

.64


In [15]:
print(read_register(SOFTWARE_P16,T_GAIN))

1488646


# Read Software register Voltage Sag, Over Current

In [16]:
print(read_register(SOFTWARE_P17,VSAG_LEVEL))
print(read_register(SOFTWARE_P17,IOVER_LEVEL))

0
16777087


# Read Instantenous voltage and current

In [17]:
instruction(CONT_CONV)

In [18]:
for i in range(10):
    print(read_register(SOFTWARE_P16,V_INST))

8394752
7345920
2103296
.10491392
5888
4199424
5248768
7345920
13637376
1054976


In [19]:
for i in range(10):
    print(read_register(SOFTWARE_P16,I_INST))

12582912
11534336
.12582912
11534336
11534336
12582912
11534336
11534336
11534336
12582912
